# Notebook for extracting data for CEP-related analysis

This script retrieves and transforms various data sources for this analysis: 

1. EIA-861 files detailing MWh, Revenue and annual customers for CEPs
2. Customer migration statistics from the Public Utilities Commission
3. Historical standard offer rates by service territory (compiled manually)

In [3]:
import pandas as pd
import requests
import numpy as np
import os
import glob
import re
from datetime import datetime
from zipfile import ZipFile
from io import BytesIO
import helper_functions as hf
from importlib import reload

## 1. Extract and store EIA files

### Set locations

In [6]:
data_dir = os.path.join(os.getcwd(), 'raw_data')
process_dir = os.path.join(os.getcwd(), 'prepared_data')

#### Read in files from ZIPs at [EIA page](https://www.eia.gov/electricity/data/eia861/)

In [147]:
hf.download_eia_861(2012, 2022, data_dir)

Extracted Sales_Ult_Cust_2012.xlsx
Extracted Sales_Ult_Cust_2013.xls
Extracted Sales_Ult_Cust_2014.xls
Extracted Sales_Ult_Cust_2015.xlsx
Extracted Sales_Ult_Cust_2016.xlsx
Extracted Sales_Ult_Cust_2017.xlsx
Extracted Sales_Ult_Cust_2018.xlsx
Extracted Sales_Ult_Cust_2019.xlsx
Extracted Sales_Ult_Cust_2020.xlsx
Extracted Sales_Ult_Cust_2021.xlsx
Reading in /Users/Darren/git-clones/data-projects/CEPs/etl_scripts/raw_data/Sales_Ult_Cust_2017.xlsx
Reading in /Users/Darren/git-clones/data-projects/CEPs/etl_scripts/raw_data/Sales_Ult_Cust_2021.xlsx
Reading in /Users/Darren/git-clones/data-projects/CEPs/etl_scripts/raw_data/Sales_Ult_Cust_2020.xlsx
Reading in /Users/Darren/git-clones/data-projects/CEPs/etl_scripts/raw_data/Sales_Ult_Cust_2016.xlsx
Reading in /Users/Darren/git-clones/data-projects/CEPs/etl_scripts/raw_data/Sales_Ult_Cust_2013.xls
Reading in /Users/Darren/git-clones/data-projects/CEPs/etl_scripts/raw_data/Sales_Ult_Cust_2012.xlsx
Reading in /Users/Darren/git-clones/data-projec

#### Transform and merge separate files, write to CSV and store as dataframe for inspection

In [84]:
eia_df = hf.process_and_merge_861(data_dir=data_dir, process_dir=process_dir)

Reading in /Users/Darren/git-clones/data-projects/CEPs/etl_scripts/raw_data/Sales_Ult_Cust_2017.xlsx
Reading in /Users/Darren/git-clones/data-projects/CEPs/etl_scripts/raw_data/Sales_Ult_Cust_2021.xlsx
Reading in /Users/Darren/git-clones/data-projects/CEPs/etl_scripts/raw_data/Sales_Ult_Cust_2020.xlsx
Reading in /Users/Darren/git-clones/data-projects/CEPs/etl_scripts/raw_data/Sales_Ult_Cust_2016.xlsx
Reading in /Users/Darren/git-clones/data-projects/CEPs/etl_scripts/raw_data/Sales_Ult_Cust_2013.xls
Reading in /Users/Darren/git-clones/data-projects/CEPs/etl_scripts/raw_data/Sales_Ult_Cust_2012.xlsx
Reading in /Users/Darren/git-clones/data-projects/CEPs/etl_scripts/raw_data/Sales_Ult_Cust_2014.xls
Reading in /Users/Darren/git-clones/data-projects/CEPs/etl_scripts/raw_data/Sales_Ult_Cust_2015.xlsx
Reading in /Users/Darren/git-clones/data-projects/CEPs/etl_scripts/raw_data/Sales_Ult_Cust_2019.xlsx
Reading in /Users/Darren/git-clones/data-projects/CEPs/etl_scripts/raw_data/Sales_Ult_Cust_20

## Extract and process 

In [83]:
reload(hf)

<module 'helper_functions' from '/Users/Darren/git-clones/data-projects/CEPs/etl_scripts/helper_functions.py'>

## 2. Maine customer migration statistics

Update file dir based on [this page](https://www.maine.gov/mpuc/regulated-utilities/electricity/choosing-supplier/migration-statistics) from the Maine PUC. 

In [109]:
migration_xls = 'https://www.maine.gov/mpuc/sites/maine.gov.mpuc/files/inline-files/Standard%20Offer%20Migration%20Stats%20through%2008.15.23.xls'

customer_df = pd.read_excel(migration_xls, skiprows=3, usecols='A:AK')

In [111]:
customer_df = customer_df[~customer_df.iloc[:, 1].isna()]
customer_df.head()

,Unnamed: 0,# CEP,% CEP,# customers in class,# CEP.1,% CEP.1,# customers in class.1,# CEP.2,% CEP.2,# customers in class.2,...,# customers in class.8,# CEP.9,% CEP.9,# customers in class.9,# CEP.10,% CEP.10,# customers in class.10,# CEP.11,% CEP.11,# customers in class.11
1,2023-07-31,10708.0,0.083539,128179.0,883.0,0.511587,1726.0,35.0,0.614035,57.0,...,36238.0,50.0,0.246305,203.0,13.0,0.812500,16.0,387.0,0.010615,36457.0
2,2023-06-30,10553.0,0.082000,128042.0,877.0,0.508000,1728.0,35.0,0.614000,57.0,...,36222.0,51.0,0.250000,204.0,13.0,0.813000,16.0,414.0,0.011000,36442.0
3,2023-05-31,10329.0,0.080830,127786.0,863.0,0.499421,1728.0,35.0,0.625000,56.0,...,36228.0,51.0,0.250000,204.0,13.0,0.812500,16.0,415.0,0.011386,36448.0
4,2023-04-30,10008.0,0.078558,127396.0,847.0,0.493302,1717.0,36.0,0.654545,55.0,...,36160.0,50.0,0.235849,212.0,13.0,0.812500,16.0,384.0,0.010553,36388.0
5,2023-03-31,9678.0,0.076214,126984.0,835.0,0.495255,1686.0,36.0,0.654545,55.0,...,36140.0,50.0,0.242718,206.0,13.0,0.812500,16.0,385.0,0.010588,36362.0
6,2023-02-28,9590.0,0.075572,126899.0,821.0,0.492797,1666.0,36.0,0.654545,55.0,...,36137.0,55.0,0.261905,210.0,13.0,0.812500,16.0,403.0,0.011083,36363.0
7,2023-01-31,9865.0,0.077760,126865.0,811.0,0.487087,1665.0,36.0,0.654545,55.0,...,36149.0,56.0,0.265403,211.0,13.0,0.812500,16.0,412.0,0.011326,36376.0
8,2022-12-31,10017.0,0.079002,126794.0,796.0,0.476647,1670.0,36.0,0.654545,55.0,...,36115.0,47.0,0.221698,212.0,6.0,0.375000,16.0,314.0,0.008640,36343.0
9,2022-11-30,10162.0,0.080170,126756.0,810.0,0.482430,1679.0,36.0,0.642857,56.0,...,36137.0,47.0,0.223810,210.0,6.0,0.375000,16.0,320.0,0.008800,36363.0
10,2022-10-31,10285.0,0.080995,126983.0,818.0,0.482027,1697.0,36.0,0.642857,56.0,...,36104.0,51.0,0.245192,208.0,6.0,0.666667,9.0,328.0,0.009031,36321.0


In [112]:
exclude_regex = f'^(?!%)'

df_cols = customer_df.filter(regex=exclude_regex)

df_cols.head(10)

,Unnamed: 0,# CEP,# customers in class,# CEP.1,# customers in class.1,# CEP.2,# customers in class.2,# CEP.3,# customers in class.3,# CEP.4,...,# CEP.7,# customers in class.7,# CEP.8,# customers in class.8,# CEP.9,# customers in class.9,# CEP.10,# customers in class.10,# CEP.11,# customers in class.11
1,2023-07-31,10708.0,128179.0,883.0,1726.0,35.0,57.0,11626.0,129962.0,74959.0,...,80902.0,636183.0,324.0,36238.0,50.0,203.0,13.0,16.0,387.0,36457.0
2,2023-06-30,10553.0,128042.0,877.0,1728.0,35.0,57.0,11465.0,129827.0,75195.0,...,81400.0,662860.0,350.0,36222.0,51.0,204.0,13.0,16.0,414.0,36442.0
3,2023-05-31,10329.0,127786.0,863.0,1728.0,35.0,56.0,11227.0,129570.0,71824.0,...,77968.0,663946.0,351.0,36228.0,51.0,204.0,13.0,16.0,415.0,36448.0
4,2023-04-30,10008.0,127396.0,847.0,1717.0,36.0,55.0,10891.0,129168.0,60836.0,...,66221.0,595513.0,321.0,36160.0,50.0,212.0,13.0,16.0,384.0,36388.0
5,2023-03-31,9678.0,126984.0,835.0,1686.0,36.0,55.0,10549.0,128725.0,66214.0,...,72105.0,663075.0,322.0,36140.0,50.0,206.0,13.0,16.0,385.0,36362.0
6,2023-02-28,9590.0,126899.0,821.0,1666.0,36.0,55.0,10447.0,128620.0,61463.0,...,66670.0,600891.0,335.0,36137.0,55.0,210.0,13.0,16.0,403.0,36363.0
7,2023-01-31,9865.0,126865.0,811.0,1665.0,36.0,55.0,10712.0,128585.0,69168.0,...,74827.0,661304.0,343.0,36149.0,56.0,211.0,13.0,16.0,412.0,36376.0
8,2022-12-31,10017.0,126794.0,796.0,1670.0,36.0,55.0,10849.0,128519.0,70693.0,...,76325.0,660258.0,261.0,36115.0,47.0,212.0,6.0,16.0,314.0,36343.0
9,2022-11-30,10162.0,126756.0,810.0,1679.0,36.0,56.0,11008.0,128491.0,68339.0,...,73693.0,625336.0,267.0,36137.0,47.0,210.0,6.0,16.0,320.0,36363.0
10,2022-10-31,10285.0,126983.0,818.0,1697.0,36.0,56.0,11139.0,128736.0,69581.0,...,74963.0,633798.0,271.0,36104.0,51.0,208.0,6.0,9.0,328.0,36321.0


In [130]:
bhe_loc = slice(1, 3)
cmp_loc = slice(9, 11)
mps_loc = slice(17, 19)

bhe_df = df_cols.iloc[: , np.r_[0, bhe_loc]].assign(UTILITY='BANGOR HYDRO DISTRICT')
cmp_df = df_cols.iloc[: , np.r_[0, cmp_loc]].assign(UTILITY='CENTRAL MAINE POWER CO.')
mps_df = df_cols.iloc[: , np.r_[0, mps_loc]].assign(UTILITY='MAINE PUBLIC SERVICE')

In [132]:
district_dict = {
    'BANGOR HYDRO DISTRICT': bhe_loc,
    'CENTRAL MAINE POWER CO.': cmp_loc,
    'MAINE PUBLIC SERVICE': mps_loc
}

In [143]:
dfs = []
cep_cols = ['DATE', 'CEP_CUSTOMERS', 'TOTAL_CUSTOMERS', 'UTILITY']

for utility, col_slice in district_dict.items():
    df_slice = df_cols.iloc[: , np.r_[0, col_slice]].assign(UTILITY=utility)
    df_slice.columns = cep_cols 
    dfs.append(df_slice)
    
migration_df = pd.concat(dfs, axis=0)

migration_df.to_csv(os.path.join(process_dir, 'mpuc_customer_migration_statistics.csv'), index=False)